In [1]:
from phylodeep import BD, BDEI, BDSS, FULL
from phylodeep.modeldeep import modeldeep
from phylodeep.paramdeep import paramdeep


path_to_tree = "/Users/mmcanear/Projects/PhD_Courses/STAT700/phylodeep/test_tree_HIV_Zurich/Zurich.trees"

# set presumed sampling probability
sampling_proba = 0.25

# NOTE: Absolutely does not work due to broadcasting issues
# a priori check for models BD, BDEI, BDSS
# checkdeep(path_to_tree, model=BD, outputfile_png='BD_a_priori_check.png')
# checkdeep(path_to_tree, model=BDEI, outputfile_png='BDEI_a_priori_check.png')
# checkdeep(path_to_tree, model=BDSS, outputfile_png='BDSS_a_priori_check.png')


# model selection
model_BDEI_vs_BD_vs_BDSS = modeldeep(
    path_to_tree, sampling_proba, vector_representation=FULL
)

# the selected model is BDSS

# parameter inference
param_BDSS = paramdeep(
    path_to_tree,
    sampling_proba,
    model=BDSS,
    vector_representation=FULL,
    ci_computation=True,
)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


# Encoding

Exploratory analysis of how to actually get the CBLV representation in Phylodeep; this is NOT strictly provided by the package, it seems.

In [ ]:
from phylodeep.encoding import encode_into_most_recent 
from ete3 import Tree
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
import re

output_dir = Path("../output_trees")

param_files = list(output_dir.glob("*.csv"))
all_tree_files = list(output_dir.glob("*.nwk"))

tree_indices = [int(re.sub("[^0-9]", "", f.name)) for f in all_tree_files]
param_indices = [int(re.sub("[^0-9]", "", f.name)) for f in param_files]


In [ ]:
sampling_proba = 0.5

def read_tree(tree_file, idx):
    try:
        with open(tree_file, "r") as f:
            nwk_str = f.readline().strip()   
        tree = Tree(nwk_str, format=1)
        tree_encoding, _ = encode_into_most_recent(tree, 0.5)
        
        tree_encoding["idx"] = idx
        tree_encoding = tree_encoding.set_index("idx")
        return tree_encoding

    except Exception as e:
        # print(f"Error reading {tree_file}: {e}")
        return None

all_tree_files = list(output_dir.glob("*.nwk"))
all_tree_outputs = [read_tree(f, idx) for f, idx in tqdm(zip(all_tree_files, tree_indices), total=len(tree_indices))]


  7%|▋         | 452/6858 [00:07<01:27, 73.09it/s]

In [ ]:
import re

def _skip_reader(file_path, idx):
    try:
        df = pd.read_csv(file_path)
        df["idx"] = idx
        df.set_index("idx", inplace=True)
        return df
    except Exception as e:
        return pd.DataFrame()


valid_params = [
    _skip_reader(f, idx) for f, idx in tqdm(zip(param_files, param_indices), total=len(param_files))
]
assert len(valid_params) == len(param_indices)
param_df = pd.concat(valid_params)
# plt.scatter(params["la_II"], params["psi_I"], s=0.5)

100%|██████████| 8656/8656 [00:07<00:00, 1100.63it/s]


,R,R_I,d,d_I,la_II,p_I,psi_I,t_I,R.1,d.1,tips,hidden_trees,end_time,R_observed,d_observed,zeta_observed
idx,,,,,,,,,,,,,,,,
5633,1.320240,1.320240,1.96464,1.96464,0.672,0.5,0.509,1.96464,1.320240,1.96464,151,1,25.9030,1.34118,1.89688,0.331140
3242,1.130980,1.130980,2.51889,2.51889,0.449,0.5,0.397,2.51889,1.130980,2.51889,170,3,29.5353,1.15361,2.44580,0.461957
7024,0.552209,0.552209,2.00803,2.00803,0.275,0.5,0.498,2.00803,0.552209,2.00803,10,2,19.7618,1.07143,3.12717,0.666667
8317,1.959130,1.959130,2.40385,2.40385,0.815,0.5,0.416,2.40385,1.959130,2.40385,125,0,15.0599,1.85106,2.36896,0.287356
1455,1.318780,1.318780,4.36681,4.36681,0.302,0.5,0.229,4.36681,1.318780,4.36681,238,0,57.7343,1.27293,3.98281,0.418278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7983,1.556000,1.556000,2.00000,2.00000,0.778,0.5,0.500,2.00000,1.556000,2.00000,56,0,12.9080,1.54962,1.88254,0.275862
829,1.747660,1.747660,2.33645,2.33645,0.748,0.5,0.428,2.33645,1.747660,2.33645,377,0,17.3760,1.73056,2.32328,0.302568
9010,1.953850,1.953850,15.38460,15.38460,0.127,0.5,0.065,15.38460,1.953850,15.38460,249,0,75.5877,1.99810,14.72210,0.237369
